In [ ]:
import cv2
import HandTrackingModule as htm
import time
import autopy
import numpy as np


In [ ]:
cap=cv2.VideoCapture(0)
wCam, hCam = 640, 480
cap.set(3,wCam)
cap.set(4,hCam)
wScr,hScr=autopy.screen.size()
#print(wScr,hScr)

In [ ]:
detector=htm.handDetector(detectionCon=0.8)
pTime=0
cTime=0
frameR=100 #frame Reduction! 
smoothening=7
# we will use to smooth value !
#previous location :
Px,Py=0,0
#current location:
Cx,Cy=0,0
while True:
    #1 find landmarks:
    succes,img=cap.read()
    img=cv2.flip(img,1)
    img=detector.findHands(img)
    #2 take hand pixel position:
    LmList,bbox=detector.findPosition(img)
    if LmList:
        x1,y1=LmList[8][1],LmList[8][2]
        x2,y2=LmList[12][1],LmList[12][2]
        #3 take fingers!
        fingers=detector.fingersUp()
        #4 moving mode!   
        if (fingers[1]==1 and fingers[2]==0) :
            #5 convert our condinate:
            #6 cerate a range to move the mouse !
            cv2.rectangle(img,(frameR,frameR),(wCam-frameR,hCam-frameR),(255,0,255),2)
            x3=np.interp(x1,(frameR,wCam-frameR),(0,wScr))
            y3=np.interp(y1,(frameR,hCam-frameR),(0,hScr))
            # 6+ --smooth the value :
            Cx=Px+(x3-Px) / smoothening
            Cy=Py+(y3-Py) / smoothening
            #7 Move the mouse :
            autopy.mouse.move(Cx,Cy)
            Px,Py=Cx,Cy
            cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
            #8 the click mouse: 
            print(fingers[1],fingers[2])
        if (fingers[1]==1 and fingers[2]==1) :
                l,img,lineInfo=detector.findDistance(8,12,img)
                print(l)
                if l<42:
                    cv2.circle(img,(lineInfo[4],lineInfo[5]),15,(0,255,0),cv2.FILLED)
                    autopy.mouse.click()

       
    #frame rate :
    cTime=time.time()
    fps=1/(cTime-pTime) 
    pTime=cTime
    #show  fps (10,70) for the position , 3 mean the skale , the 1 for letter ( big , small )
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),1)
    cv2.imshow("Image",img)
    cv2.waitKey(1)